In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from keras.preprocessing.image import load_img, img_to_array

import keras
from keras.layers import Dense, Conv2D, MaxPooling2D,Flatten, Dropout, Activation, Input
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
#print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

## 1. Load Train DataSet

In [11]:
#### Load training Data CSV
trainDF = pd.read_csv('../input/train.csv')

#### Get training images 
trainImgList = list(trainDF['id'])
trainImg = []

for img in trainImgList:
    originalImage = load_img(f'../input/train/train/{img}')
    arrayImage = img_to_array(originalImage) / 255
    trainImg.append(arrayImage)
    
trainImgNP = np.array(trainImg)

#### Get training Lables
trainLabels = trainDF['has_cactus'].values

## 2. Model Helper Functions

In [12]:
#### Model Helper Functions
def getConvLayer(inputLayer, kernelSize = 2 ,filters = 64):
    conv1 = Conv2D(filters, kernelSize, activation = 'relu')(inputLayer)
    #conv2 = Conv2D(filters, kernelSize, activation = 'relu')(conv1)
    pool1 = MaxPooling2D()(conv1)
    return pool1

def getFlattenLayer(inputLayer):
    flat = Flatten()(inputLayer)
    return flat

def getDenseLayer(inputLayer, units = 512, rate = .5):
    dense1 = Dense(units, activation = 'relu')(inputLayer)
    drop1 = Dropout(rate)(dense1)
    return drop1

def getOutLayer(inputLayer):
    out1 = Dense(1, activation = 'sigmoid')(inputLayer)
    return out1

def getModel():
    #### Create Model and show Summary
    inputLayer = Input(shape = [32, 32, 3])

    # 1st ConvLayer
    conv1 = getConvLayer(inputLayer)

    # 2nd ConvLayer
    conv2 = getConvLayer(conv1, filters = 32)
    
    # 3rd ConvLayer
    conv3 = getConvLayer(conv2, filters = 16)

    # FlattenLayer
    flat = getFlattenLayer(conv3)

    # 1st DenseLayer
    dense1 = getDenseLayer(flat)

    # 2nd DenseLayer
    dense2 = getDenseLayer(dense1)

    # 3rd DenseLayer
    dense3 = getDenseLayer(dense2)

    # OutputLayer
    out = getOutLayer(dense3)

    # Create Model
    model = Model(inputLayer, out)
    #model.summary()
    
    return model

#### Learn Rate reduction
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5, min_lr=0.00001)

## 3. Train Model

In [13]:
#### Compile and train Model
model = getModel()

model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])

model.fit(x = trainImgNP, y = trainLabels, batch_size = 64, epochs = 100, validation_split = .1, callbacks=[learning_rate_reduction])

Train on 15750 samples, validate on 1750 samples
Epoch 1/100
15750/15750 [==============================] - 3s 168us/step - loss: 0.4257 - acc: 0.7858 - val_loss: 0.2462 - val_acc: 0.9263
Epoch 2/100
15750/15750 [==============================] - 2s 119us/step - loss: 0.1659 - acc: 0.9406 - val_loss: 0.1349 - val_acc: 0.9491
Epoch 3/100
15750/15750 [==============================] - 2s 119us/step - loss: 0.1261 - acc: 0.9539 - val_loss: 0.1255 - val_acc: 0.9520
Epoch 4/100
15750/15750 [==============================] - 2s 119us/step - loss: 0.1095 - acc: 0.9601 - val_loss: 0.0843 - val_acc: 0.9697
Epoch 5/100
15750/15750 [==============================] - 2s 118us/step - loss: 0.0870 - acc: 0.9694 - val_loss: 0.0674 - val_acc: 0.9754
Epoch 6/100
15750/15750 [==============================] - 2s 118us/step - loss: 0.0788 - acc: 0.9711 - val_loss: 0.0602 - val_acc: 0.9777
Epoch 7/100
15750/15750 [==============================] - 2s 118us/step - loss: 0.0696 - acc: 0.9754 - val_loss: 0.0

15750/15750 [==============================] - 2s 119us/step - loss: 0.0185 - acc: 0.9937 - val_loss: 0.0305 - val_acc: 0.9891
Epoch 58/100
15750/15750 [==============================] - 2s 120us/step - loss: 0.0187 - acc: 0.9943 - val_loss: 0.0292 - val_acc: 0.9891
Epoch 59/100
15750/15750 [==============================] - 2s 119us/step - loss: 0.0191 - acc: 0.9933 - val_loss: 0.0283 - val_acc: 0.9886
Epoch 60/100
15750/15750 [==============================] - 2s 119us/step - loss: 0.0189 - acc: 0.9931 - val_loss: 0.0301 - val_acc: 0.9886
Epoch 61/100
15750/15750 [==============================] - 2s 119us/step - loss: 0.0190 - acc: 0.9940 - val_loss: 0.0284 - val_acc: 0.9891
Epoch 62/100
15750/15750 [==============================] - 2s 121us/step - loss: 0.0169 - acc: 0.9947 - val_loss: 0.0330 - val_acc: 0.9886
Epoch 63/100
15750/15750 [==============================] - 2s 119us/step - loss: 0.0183 - acc: 0.9937 - val_loss: 0.0283 - val_acc: 0.9897
Epoch 64/100
15750/15750 [=======

## 4. Load Test DataSet

In [14]:
#### Get testing images 
testImgList = os.listdir("../input/test/test")
testImg = []

for img in testImgList:
    originalImage = load_img(f'../input/test/test/{img}')
    arrayImage = img_to_array(originalImage) / 255
    testImg.append(arrayImage)
    
testImgNP = np.array(testImg)

## 5. Predict Labels

In [15]:
#### get Predictions for test Data
testPred = model.predict(testImgNP)
testPredRound = np.round(testPred)
subPred = [int(x) for [x] in testPredRound]

## 6. Submission

In [16]:
#### Create a Submission DF
sub = pd.DataFrame({'id' : testImgList, 'has_cactus' : subPred})
sub.to_csv('submission_3C_3D.csv', index = False)